In [1]:
import matplotlib as plt
import pandas as pd
import numpy as np

import sklearn
import sklearn.feature_extraction.text as sklearnText

In [2]:
train_set = pd.read_csv("Data/train_data.csv")
train_labels = pd.read_csv("Data/train_labels.csv")

In [3]:
train_set = pd.merge(train_set, train_labels, on='id')

In [4]:
train_set

,id,question1,question2,is_duplicate_x,is_duplicate_y
0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,NaN,0
1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,NaN,0
2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,NaN,0
3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,NaN,0
4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,NaN,0
5,5,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",NaN,1
6,6,Should I buy tiago?,What keeps childern active and far from phone ...,NaN,0
7,7,How can I be a good geologist?,What should I do to be a great geologist?,NaN,1
8,8,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",NaN,0
9,9,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,NaN,0


In [5]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet as wn
import itertools

In [6]:
train_set[train_set['id'] == 1]['question1']

1    What is the story of Kohinoor (Koh-i-Noor) Dia...
Name: question1, dtype: object

In [7]:
#use stemmer 
def sentenceTokenize(question1, question2):
    stm = PorterStemmer()
    sent1 = question1 
    sent2 = question2
    #Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}

    s1 = nltk.pos_tag(nltk.word_tokenize(sent1))

    s1 = dict(filter(lambda x: len(x[1])>0,
                     map(lambda row: (row[0],wn.synsets(
                           stm.stem(row[0]),
                           tag_dict[row[1][0]])) if row[1][0] in tag_dict.keys() 
                         else (row[0],[]),s1)))

    s2 = nltk.pos_tag(nltk.word_tokenize(sent2))

    s2 = dict(filter(lambda x: len(x[1])>0,
                     map(lambda row: (row[0],wn.synsets(
                              stm.stem(row[0]),
                              tag_dict[row[1][0]])) if row[1][0] in tag_dict.keys() 
                         else (row[0],[]),s2)))
    
    return [s1,s2]

In [8]:
def calc_sim(sentenceList):
    res = {}
    s1 = sentenceList[0]
    s2 = sentenceList[1]
    for w2,gr2 in s2.items():
        for w1,gr1 in s1.items():
            tmp = pd.Series(list(map(lambda row: row[1].path_similarity(row[0]),
                                     itertools.product(gr1,gr2)))).dropna()
            if len(tmp)>0:
                res[(w1,w2)] = tmp.max()
    similarity = pd.Series(res).groupby(level=0).max().mean()
    return similarity

In [9]:
#train_set['similarity'] = train_set[calc_sim(sentenceTokenize(str(train_set['question1']), str(train_set['question2'])))]

for index, row in train_set.iterrows():
    similarity = calc_sim(sentenceTokenize(str(row['question1']), str(row['question2'])))
    train_set.loc[index, 'similarity'] = similarity


    

KeyboardInterrupt: 

In [ ]:
train_set = train_set[:62851]
train_set

In [ ]:
train_set.to_csv('data/similarity.csv')